# 1. Обработка исключений

`try` пытается что-то сделать, если вылезает исключение, переходит в `except`

In [84]:
x = [1, 2, 3, 0, -2, "a", 2]

for item in x: 
    try:
        print(100/item)
        
    except ZeroDivisionError:
        print("Деление на ноль")
        
    except TypeError:
        print("Типы несовместимы")
        
    # общий эксцепт, который ловит любые исключения
    except:
        print("Что-то пошло не так...")
        

100.0
8
50.0
8
33.333333333333336
8
Деление на ноль
8
-50.0
8
Типы несовместимы
8
50.0
8


`raise` вызывет исключение по ходу кода

In [82]:
a,b = int(input()),int(input())  # вводим 1 затем 0

if b == 0:
    raise ZeroDivisionError
else:
    print(a/b)

1
0


ZeroDivisionError: 

`assert` - санитарная проверка 

In [85]:
assert(True)
#  код работает дальше

In [86]:
assert(1==0)

AssertionError: 

In [87]:
# у функции два аргумента, чтобы описывать проблемы
assert  False, "Это проблема"

AssertionError: Это проблема

свои собственные исключения

In [55]:
class MyError(Exception):
    print("Это проблема")

raise MyError("ошибка MyError")

Это проблема


MyError: ошибка MyError

### Исходники: 

* https://pythonru.com/osnovy/obrabotka-iskljuchenij-python-blok-try-except-blok-finally

# 2. Итераторы и генераторы

Словарик: 

* __Итератор__ - объект перечислитель, который выводит каждый элемент по очереди
* __Генератор__ - подвид итераторов, который перебирает элементы, но не индексирует их (грубо говоря это просто функция с `yield` вместо `return`

Примеры встроенных генераторов: `enumerate` и `range`

Итерирование - просто тупо перебор. 

In [88]:
num_list = [11, 22, 33]

for j,item in enumerate(num_list):
    print(j, item)

0 11
1 22
2 33


Можно переписать это через методы `iter` и `next`:

In [89]:
itr = iter(num_list)

In [90]:
next(itr)

11

In [91]:
next(itr)

22

In [92]:
next(itr)

33

In [93]:
next(itr)

StopIteration: 

Когда кончились объекты, выскакивает исключение `StopIteration`. Цикл обрабатывает это исключение незаметно для нас. 

Можно придумать свой итератор. Для этого надо написать класс с двумя методами `__iter__()` и  `__next__()`.

* __iter__ возвращает объект для итерирования 

* __next__ озвращает новые элементы по ходу итерирования

In [94]:
for i in range(10, 15):
    print(i)

10
11
12
13
14


In [95]:
class my_range:
    
    def __init__(self, low, high):
        self.current = low - 1
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        self.current += 1
        if self.current < self.high:
            return self.current
        else:
            raise StopIteration

In [96]:
for c in my_range(100, 105):
    print(c)

100
101
102
103
104


Напишем генератор для чисел Фиббоначи. В вариации с `yield` и без. 

$$
a_1 = 0, a_2 = 1, a_3 = a_1 + a_2, a_4 = a_3 + a_2
$$

In [107]:
class FibonacciGenerator:
    
    def __init__(self):
        self.prev = 0
        self.cur = 1
        
    def __iter__(self):
        return self

    def __next__(self):
        result = self.prev
        self.prev, self.cur = self.cur, self.prev + self.cur
        return result


for i in FibonacciGenerator():
    print(i)
    if i > 100:
        break

0
1
1
2
3
5
8
13
21
34
55
89
144


In [ ]:
gen = FibonacciGenerator()
while True:
    i = gen.__next__()
    print(i)
    if i > 100:
        break

Используя `yield` можно сильно упростить реализацию и переписать генератор в виде функции. 

In [104]:
def fibonacci():
    
    prev, cur = 0, 1
    
    while True:
        yield prev
        prev, cur = cur, prev + cur


for i in fibonacci():
    print(i)
    if i > 100:
        break

0
1
1
2
3
5
8
13
21
34
55
89
144


Про то как работает `yield`:

In [105]:
def gen_fun():
    
    print('block 1')
    yield 1
    
    print('block 2')
    yield 2
    
    print('end')

for i in gen_fun():
    print(i)

block 1
1
block 2
2
end


__Происходит следующее:__

1. при вызове функции __gen_fun__ создается объект-генератор
2. __for__ вызывает __iter()__ с этим объектом и получает итератор этого генератора
3. в цикле вызывает функция __next()__ с этим итератором пока не будет получено исключение __StopIteration__
4. при каждом вызове __next__ выполнение в функции начинается с того места где было завершено в последний раз и продолжается до следующего __yield__

In [ ]:
def gen_fun_1():
    print('block 1')
    return 1

def gen_fun_2():
    print('block 2')
    return 2

def gen_fun_3():
    print('end')

def gen_fun_end():
    raise StopIteration

Когда интерпретатор доходит до ключевого слова `return`, выполнение функции полностью прекращается. Но когда он доходит до ключевого слова `yield`, программа приостанавливает выполнение функции и возвращает значение в итерируемый объект. После этого интерпретатор возвращается к генератору, чтобы повторить процесс для нового значения.

Можно написать свой собственный `range`:

In [108]:
def cool_range(start, stop, inc):
    x = start
    while x < stop:
        yield x
        x += inc

for n in cool_range(1, 5, 0.5):
    print(n)

1
1.5
2.0
2.5
3.0
3.5
4.0
4.5


## Генераторные выражения

In [109]:
[i**2 for i in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [26]:
(i**2 for i in range(10))

<generator object <genexpr> at 0x11109e750>

* Первый вариант создаст сразу целый список, со всеми вытекающими от сюда проблемами.
* Второй вариант работает схожим с нашей функцией cool_range образом и может генерировать без проблем любой диапазон.

## Yield from

Для обхода ограниченно вложенных структур, традиционный подход использовать вложенные циклы. Тот же подход можно использовать когда генераторная функция должна отдавать значения, порождаемые другим генератором.

In [115]:
# chain - итератор, который объединит несколько итераторв в один общий 

def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

g = chain([1, 2, 3], {'A', 'B', 'C'}, '...')
list(g)

[1, 2, 3, 'A', 'B', 'C', '.', '.', '.', 'n', 'j', 'f', 'e', 'w', 'e']

Но вложенные циклы можно убрать, добавив конструкцию yield from:


In [116]:
def chain(*iterables):
    for it in iterables:
        yield from it

g = chain([1, 2, 3], {'A', 'B', 'C'}, '...')
print(list(g))

[1, 2, 3, 'A', 'B', 'C', '.', '.', '.']


Основная польза yield from в создании прямого канала между внутренним генератором и клиентом внешнего генератора.

### Пример:  свой собственный flatten

In [120]:
x = [1,2,3]
isinstance(x, Iterable)

True

In [121]:
x = 555
isinstance(x, Iterable)

False

In [122]:
x = "Маша"
isinstance(x, Iterable)

True

In [123]:
x = "Маша"
isinstance(x, str)

True

In [135]:
from collections import Iterable

def flatten(items, ignore_types=(str, bytes)):
    """
      str, bytes - являются итерируемыми объектами,
       но их хотим возвращать целыми
    """
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x

items = [1, 2, [3, 4, [5, [[6]]], 7], 8, ('A', {'Маша', 'C'}), range(10,15)]

list(flatten(items))

[1, 2, 3, 4, 5, 6, 7, 8, 'A', 'Маша', 'C', 10, 11, 12, 13, 14]

In [136]:
from collections import Iterable

def flatten(items, ignore_types=(bytes)):
    """
      str, bytes - являются итерируемыми объектами,
       но их хотим возвращать целыми
    """
    for x in items:
        if isinstance(x, str)  and  len(x) == 1:
            yield x
        elif isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x

items = [1, 2, [3, 4, [5, [[6]]], 7], 8, ('A', {'Маша', 'C'}), range(10,15)]

list(flatten(items))

[1, 2, 3, 4, 5, 6, 7, 8, 'A', 'М', 'а', 'ш', 'а', 'C', 10, 11, 12, 13, 14]

### Отступление: немного про переменное число аргументов

In [28]:
def print_names(name1, name2, name3):
    print("имя 1:", name1)
    print("имя 2:", name2)
    print("имя 3:", name3)

In [29]:
print_names(*['Коля', 'Маша', 'Витя'])

имя 1: Коля
имя 2: Маша
имя 3: Витя


In [30]:
name_dic = {"name3": "Витя", "name2": "Маша"}
print_names("Коля", **name_dic)

имя 1: Коля
имя 2: Маша
имя 3: Витя


__Важно:__ в модулях `collections` и `itertools` есть много полезных итераторов и генераторов уже написанных за вас. 

### Исходники: 

* https://habr.com/ru/post/337314/
* https://habr.com/ru/post/50026/
* https://anandology.com/python-practice-book/iterators.html

__Задание:__ 

Пройти  8 и 9 недели курса по python на курсере: https://www.coursera.org/learn/python-osnovy-programmirovaniya

Там можно попрактиковаться в решении простых задачек на классы и функциональное программирование (всякие встроенные итераторы и тп) 

# 3. Итерирование по файлу

Исходник: https://pyneng.readthedocs.io/ru/latest/book/17_serialization/1_csv.html

In [139]:
import csv

data = [['hostname', 'vendor', 'model', 'location'],
        ['sw1', 'Cisco', '3750', 'London, Best str'],
        ['sw2', 'Cisco', '3850', 'Liverpool, Better str'],
        ['sw3', 'Cisco', '3650', 'Liverpool, Better str'],
        ['sw4', 'Cisco', '3650', 'London, Best str']]

with open('sw_data_new.csv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    for row in data:
        writer.writerow(row)

Обратите внимание на интересную особенность: строки в последнем столбце взяты в кавычки, а остальные значения - нет.

Так получилось из-за того, что во всех строках последнего столбца есть запятая. И кавычки указывают на то, что именно является целой строкой. Когда запятая находится в кавычках, модуль csv не воспринимает её как разделитель.

In [140]:
!cat sw_data_new.csv

Итератор для считывания данных. Слово `with` это [менеджер контекста.](https://pythonworld.ru/osnovy/with-as-menedzhery-konteksta.html)

In [142]:
with open('sw_data_new.csv') as f:
    reader = csv.DictReader(f, delimiter="\t")
    for row in reader:
        print(row)
        print(row['hostname'], row['model'])

OrderedDict([('hostname', 'sw1'), ('vendor', 'Cisco'), ('model', '3750'), ('location', 'London, Best str')])
sw1 3750
OrderedDict([('hostname', 'sw2'), ('vendor', 'Cisco'), ('model', '3850'), ('location', 'Liverpool, Better str')])
sw2 3850
OrderedDict([('hostname', 'sw3'), ('vendor', 'Cisco'), ('model', '3650'), ('location', 'Liverpool, Better str')])
sw3 3650
OrderedDict([('hostname', 'sw4'), ('vendor', 'Cisco'), ('model', '3650'), ('location', 'London, Best str')])
sw4 3650


То же самое без конструкции `with`: 

In [149]:
f = open('sw_data_new.csv')
reader = csv.DictReader(f, delimiter="\t")

In [150]:
next(reader)

OrderedDict([('hostname', 'sw1'),
             ('vendor', 'Cisco'),
             ('model', '3750'),
             ('location', 'London, Best str')])

In [151]:
next(reader)

OrderedDict([('hostname', 'sw2'),
             ('vendor', 'Cisco'),
             ('model', '3850'),
             ('location', 'Liverpool, Better str')])

In [152]:
f.close()  # когда файл закрыт, новую строчку из него не достать :)

In [153]:
next(reader)

ValueError: I/O operation on closed file.